In [28]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound, Forbidden
import pandas as pd

PROJECT_ID = "charged-city-421819"
DESIRED_LOCATION = "EU"        # <-- choose "EU" or "US" once
DATASET = "covid"
TABLE = "owid"

DATASET_FQN = f"{PROJECT_ID}.{DATASET}"
TABLE_FQN   = f"{DATASET_FQN}.{TABLE}"

client = bigquery.Client(project=PROJECT_ID)

# 1) Ensure dataset exists; capture its actual location
try:
    ds = client.get_dataset(DATASET_FQN)
    LOCATION = ds.location
    print(f"Dataset exists: {DATASET_FQN} | location: {LOCATION}")
except NotFound:
    ds = bigquery.Dataset(DATASET_FQN)
    ds.location = DESIRED_LOCATION
    try:
        ds = client.create_dataset(ds)                 # returns Dataset
        LOCATION = ds.location
        print(f"Created dataset: {DATASET_FQN} | location: {LOCATION}")
    except Forbidden:
        raise SystemExit(
            "No permission to create dataset. Skip the 'own table' path and use the public dataset (your main notebook already works)."
        )

# 2) Ensure table exists; if not, create it with a small OWID sample
try:
    client.get_table(TABLE_FQN)
    print(f"Table exists: {TABLE_FQN}")
except NotFound:
    print("Table not found; creating with a small sample…")
    url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
    usecols = [
        "iso_code","location","date","population",
        "total_cases","new_cases","total_deaths","new_deaths",
        "people_vaccinated","people_fully_vaccinated","total_boosters"
    ]
    df = pd.read_csv(url, usecols=usecols, parse_dates=["date"], nrows=100_000)
    df["date"] = df["date"].dt.date
    num_cols = [c for c in df.columns if c not in ("iso_code","location","date")]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE", autodetect=True)
    client.load_table_from_dataframe(df, TABLE_FQN, job_config=job_config).result()
    print("Created table with rows:", client.get_table(TABLE_FQN).num_rows)


Dataset exists: charged-city-421819.covid | location: EU
Table exists: charged-city-421819.covid.owid


In [29]:
ddl = f"CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.covid` OPTIONS(location='{LOCATION}')"
client.query(ddl, location=LOCATION).result()
print("Schema ready:", f"{PROJECT_ID}.covid")

# sanity: list datasets we see in the project
print("Datasets:", [d.dataset_id for d in client.list_datasets(PROJECT_ID)])

Schema ready: charged-city-421819.covid
Datasets: ['covid']


In [30]:
from google.cloud import bigquery
import pandas as pd

TABLE_ID = f"{PROJECT_ID}.covid.owid"   # points inside the schema we just created

url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
usecols = [
    "iso_code","location","date","population",
    "total_cases","new_cases","total_deaths","new_deaths",
    "people_vaccinated","people_fully_vaccinated","total_boosters"
]

first = True
total_rows = 0
for chunk in pd.read_csv(url, usecols=usecols, parse_dates=["date"], chunksize=200_000):
    # Normalize dtypes for BQ
    chunk["date"] = chunk["date"].dt.date
    num_cols = [c for c in chunk.columns if c not in ("iso_code","location","date")]
    chunk[num_cols] = chunk[num_cols].apply(pd.to_numeric, errors="coerce")

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE" if first else "WRITE_APPEND",
        autodetect=True,
    )

    # IMPORTANT: do NOT pass a different location here; dataset location controls it
    job = client.load_table_from_dataframe(chunk, TABLE_ID, job_config=job_config)
    job.result()

    first = False
    total_rows += len(chunk)

print("Loaded rows (from pandas):", total_rows)
print("BQ table rows (server):", client.get_table(TABLE_ID).num_rows)

Loaded rows (from pandas): 429435
BQ table rows (server): 429435


In [31]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound, Forbidden
import pandas as pd

PROJECT_ID = "charged-city-421819"
DATASET_ID = "covid"
TABLE_ID   = "owid"
DATASET_REF = f"{PROJECT_ID}.{DATASET_ID}"
TABLE_REF   = f"{DATASET_REF}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

# 1) Ensure dataset exists & capture its LOCATION
try:
    ds = client.get_dataset(DATASET_REF)
    LOCATION = ds.location
    print("Dataset exists:", DATASET_REF, "| location:", LOCATION)
except NotFound:
    print("Dataset not found; creating in US…")
    ds = bigquery.Dataset(DATASET_REF)
    ds.location = "US"
    try:
        client.create_dataset(ds)  # will fail if you lack permission
        LOCATION = "US"
        print("Created dataset:", DATASET_REF, "| location:", LOCATION)
    except Forbidden:
        raise SystemExit(
            "No permission to create dataset. Skip your own table and keep using the public dataset "
            "(your main notebook already works)."
        )

# 2) Ensure table exists; if not, create it by loading a small sample
try:
    client.get_table(TABLE_REF)
    print("Table exists:", TABLE_REF)
except NotFound:
    print("Table not found; importing a small sample to create it…")
    url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
    usecols = [
        "iso_code","location","date","population",
        "total_cases","new_cases","total_deaths","new_deaths",
        "people_vaccinated","people_fully_vaccinated","total_boosters"
    ]
    df = pd.read_csv(url, usecols=usecols, parse_dates=["date"], nrows=100_000)
    df["date"] = df["date"].dt.date
    num_cols = [c for c in df.columns if c not in ("iso_code","location","date")]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE", autodetect=True)
    client.load_table_from_dataframe(df, TABLE_REF, job_config=job_config).result()
    print("Created table with rows:", client.get_table(TABLE_REF).num_rows)

# 3) Sanity query (note: use LOCATION detected above)
sql = f"""
SELECT location, DATE(date) AS d, new_cases, total_cases, population
FROM `{TABLE_REF}`
WHERE location IN ('Serbia','Norway','Germany')
  AND date BETWEEN DATE('2020-03-01') AND DATE('2022-12-31')
ORDER BY location, d
LIMIT 10
"""
df_check = client.query(sql, location=LOCATION).result().to_dataframe()
df_check


Dataset exists: charged-city-421819.covid | location: EU
Table exists: charged-city-421819.covid.owid


,location,d,new_cases,total_cases,population
0,Germany,2020-03-01,139.0,170.0,83369840
1,Germany,2020-03-02,0.0,170.0,83369840
2,Germany,2020-03-03,0.0,170.0,83369840
3,Germany,2020-03-04,0.0,170.0,83369840
4,Germany,2020-03-05,0.0,170.0,83369840
5,Germany,2020-03-06,0.0,170.0,83369840
6,Germany,2020-03-07,0.0,170.0,83369840
7,Germany,2020-03-08,905.0,1075.0,83369840
8,Germany,2020-03-09,0.0,1075.0,83369840
9,Germany,2020-03-10,0.0,1075.0,83369840


In [32]:
sql = f"""
SELECT location, DATE(date) AS d, new_cases, total_cases, population
FROM `{TABLE_FQN}`
WHERE location IN ('Serbia','Norway','Germany')
  AND date BETWEEN DATE('2020-03-01') AND DATE('2022-12-31')
ORDER BY location, d
LIMIT 10
"""
df_check = client.query(sql, location=LOCATION).result().to_dataframe()
df_check


,location,d,new_cases,total_cases,population
0,Germany,2020-03-01,139.0,170.0,83369840
1,Germany,2020-03-02,0.0,170.0,83369840
2,Germany,2020-03-03,0.0,170.0,83369840
3,Germany,2020-03-04,0.0,170.0,83369840
4,Germany,2020-03-05,0.0,170.0,83369840
5,Germany,2020-03-06,0.0,170.0,83369840
6,Germany,2020-03-07,0.0,170.0,83369840
7,Germany,2020-03-08,905.0,1075.0,83369840
8,Germany,2020-03-09,0.0,1075.0,83369840
9,Germany,2020-03-10,0.0,1075.0,83369840


In [33]:
print("Datasets:", [d.dataset_id for d in client.list_datasets(PROJECT_ID)])


Datasets: ['covid']


In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project="charged-city-421819")

# Try to fetch the dataset directly
try:
    ds = client.get_dataset("charged-city-421819.covid")
    print("Found dataset:", ds.full_dataset_id, "| location:", ds.location)
except Exception as e:
    print("get_dataset error:", e)

# List all datasets you can see (with locations)
info = []
for d in client.list_datasets("charged-city-421819"):
    loc = client.get_dataset(d).location
    info.append((d.dataset_id, loc))
print("Datasets:", info)


In [ ]:
LOCATION = "EU"   # set to the dataset's location
# For queries:
df = client.query("SELECT 1 AS ok", location=LOCATION).result().to_dataframe()
# For loads: do NOT override location; the dataset's location is used automatically.
